In [1]:
# This notebook has been designed to run in Google Colab
!pip install torch transformers requests tqdm

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
#Import packages here
import torch
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, Dataset
from torch import nn
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import pandas as pd
import requests
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, BertTokenizer
import torch
import numpy as np
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import json
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load E. coli dataset
df = pd.read_csv("/content/drive/MyDrive/colab_data/antibertotics_data/fullseq_microbigge_ecoli.csv")


In [5]:
df.head()

,#Scientific name,Protein,BioSample,Isolate,Contig,Start,Stop,Strand,Element symbol,Element name,Type,Scope,Subtype,Class,Subclass,Method,% Coverage of reference,% Identity to reference,full_sequence,SMILES
0,Escherichia coli,HAI1959077.1,SAMN02368173,PDT000002365.3,DABDQO010000001.1,24288,24620,+,emrE,multidrug efflux SMR transporter EmrE,STRESS,plus,BIOCIDE,EFFLUX,EFFLUX,BLASTP,100.0,98.18,agattcaattggtcaacgcaacagttatgtgaaaacatggggttgc...,NaN
1,Escherichia coli,HAI1968857.1,SAMN02368174,PDT000002366.3,DABDQN010000004.1,107535,108668,+,blaEC,BlaEC family class C beta-lactamase,AMR,plus,AMR,BETA-LACTAM,BETA-LACTAM,EXACTP,100.0,100.00,tgtagcatctttgtcgagataaagaaaacagtccccatcccaatgg...,C1CNC1=O
2,Escherichia coli,HAI2018706.1,SAMN02368180,PDT000002367.3,DABDQR010000004.1,132370,132702,-,emrE,multidrug efflux SMR transporter EmrE,STRESS,plus,BIOCIDE,EFFLUX,EFFLUX,BLASTP,100.0,98.18,taaacccccgggcgcgttcctttatcaagatgacgttcgaatattc...,NaN
3,Escherichia coli,HAI2034704.1,SAMN02368181,PDT000002368.3,DABDQQ010000021.1,60473,60907,+,pcoE,copper resistance system metallochaperone PcoE,STRESS,plus,METAL,COPPER,COPPER,BLASTP,100.0,92.36,NaN,[Cu]
4,Escherichia coli,HAI2046923.1,SAMN02368184,PDT000002369.3,DABDQS010000004.1,108137,108403,+,ariR,biofilm/acid-resistance regulator AriR,STRESS,plus,ACID,NaN,NaN,EXACTP,100.0,100.00,aggatcttcaggaatacggcataaatcccaatgcctttttcaaaat...,CC(=O)NC1C(CC(OC1C(C(CO)O)O)(C(=O)O)O)O


In [6]:
df= df.head(100)

In [7]:

# Assuming your dataframe is named df
filtered_df = df.dropna(subset=['SMILES', 'full_sequence'])
filtered_df = filtered_df.reset_index(drop=True)


In [8]:
filtered_df.shape

(78, 20)

In [9]:
# Encode the 'Type' column
type_encoder = LabelEncoder()
encoded_types = type_encoder.fit_transform(filtered_df['Type'])


In [11]:
# Now add the encoded types to your dataframe or use them directly
filtered_df['encoded_Type'] = encoded_types

In [12]:
# Convert encoded types to tensors
encoded_types_tensor = torch.tensor(encoded_types).unsqueeze(1).float()

In [13]:
# Extract SMILES and sequence data
smiles_data = filtered_df['SMILES'].tolist()
sequences = filtered_df['full_sequence'].tolist()

In [14]:
# Load tokenizers
smiles_tokenizer = RobertaTokenizer.from_pretrained('DeepChem/ChemBERTa-77M-MTR', do_lower_case=False)
dna_tokenizer = BertTokenizer.from_pretrained('zhihan1996/DNA_bert_6')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/6.96k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/28.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

In [15]:
# Initialize lists to store tokenized outputs
smiles_inputs = []
dna_inputs = []

In [16]:
# Tokenize SMILES strings with progress bar
for smile in tqdm(smiles_data, desc="Tokenizing SMILES"):
    encoded_smile = smiles_tokenizer(smile, padding='max_length', truncation=True, return_tensors='pt')
    smiles_inputs.append(encoded_smile)

Tokenizing SMILES:   0%|          | 0/78 [00:00<?, ?it/s]

In [17]:
# Tokenize DNA sequences with progress bar
for sequence in tqdm(sequences, desc="Tokenizing DNA sequences"):
    encoded_sequence = dna_tokenizer(sequence, padding='max_length', truncation=True, return_tensors='pt')
    dna_inputs.append(encoded_sequence)

Tokenizing DNA sequences:   0%|          | 0/78 [00:00<?, ?it/s]

In [18]:
# Load models
smiles_model = RobertaForSequenceClassification.from_pretrained('DeepChem/ChemBERTa-77M-MTR')
dna_model = BertModel.from_pretrained('zhihan1996/DNA_bert_6')

pytorch_model.bin:   0%|          | 0.00/14.0M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


pytorch_model.bin:   0%|          | 0.00/359M [00:00<?, ?B/s]

In [19]:
# Extract embeddings for DNA sequences and SMILES data
with torch.no_grad():
    # Iterate over the list of encoded SMILES tensors
    smiles_embeddings_list = []
    for encoded_smile in smiles_inputs:
        # Access the logits (classification scores) instead of last_hidden_state
        smiles_embeddings_list.append(smiles_model(**encoded_smile).logits)

    # Stack the embeddings into a single tensor
    smiles_embeddings = torch.stack(smiles_embeddings_list)

    # Iterate over each encoded DNA sequence and get embeddings
    dna_embeddings_list = []
    for encoded_sequence in dna_inputs:
        dna_embeddings_list.append(dna_model(**encoded_sequence).last_hidden_state[:, 0, :])

    # Stack the embeddings into a single tensor
    dna_embeddings = torch.stack(dna_embeddings_list)

In [20]:
smiles_embeddings.shape

torch.Size([78, 1, 199])

In [21]:
dna_embeddings.shape

torch.Size([78, 1, 768])

In [22]:
encoded_types_tensor.shape

torch.Size([78, 1])

In [23]:
# Add an additional dimension to encoded_types_tensor
encoded_types_tensor = encoded_types_tensor.unsqueeze(2)  # Now it will have shape [78, 1, 1]

In [24]:
# Concatenate the SMILES, DNA embeddings, and the encoded 'Type' tensor
combined_embeddings = torch.cat((smiles_embeddings, dna_embeddings, encoded_types_tensor), dim=2)


In [66]:
class ResistancePredictionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(ResistancePredictionModel, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_dim, 1)  # Output layer

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten if needed (combined_embeddings should be [78, 968])
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = x.unsqueeze(2)  # Ensure output is [78, 1, 1]
        return x

In [67]:
# Initialize the model
model = ResistancePredictionModel(input_dim=968, hidden_dim=128)

In [72]:
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

In [69]:
combined_embeddings.shape

torch.Size([78, 1, 968])

In [70]:
encoded_types_tensor.shape

torch.Size([78, 1, 1])

In [73]:
# Training loop
for epoch in range(100):
    optimizer.zero_grad()

    outputs = model(combined_embeddings)

    # Compute loss
    loss = criterion(outputs, encoded_types_tensor)

    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 0.6896013617515564
Epoch 11, Loss: 0.6807441711425781
Epoch 21, Loss: 0.6589828133583069
Epoch 31, Loss: 0.6297972202301025
Epoch 41, Loss: 0.6039426922798157
Epoch 51, Loss: 0.566010594367981
Epoch 61, Loss: 0.5166274309158325
Epoch 71, Loss: 0.4547122120857239
Epoch 81, Loss: 0.38147276639938354
Epoch 91, Loss: 0.29611819982528687
